# PM2.5 forecasting

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import optuna # Hyperparameter optimization
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler # Data normalization
from sklearn.preprocessing import MinMaxScaler

/Users/valeriagonzalez/Documents/aqi/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valeriagonzalez/Documents/aqi/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [13]:
filepath = "datasets/semadet-tlaquepaque-2023-interpolated.csv"
df = pd.read_csv(filepath, parse_dates=[0], index_col=0)

## Select features and pollutant to predict

In [15]:
features = ["pm25", "tmp", "rh", "ws", "wd"]
pollutant = "pm25"

In [16]:
def select_df_features(df:pd.DataFrame, features:list):
    df_select = pd.DataFrame()
    for feature in features:
        df_select[feature] = df[feature]
    return df_select

In [17]:
df_select = select_df_features(df, features)

In [22]:
df.head(3)

,o3,pm10,pm25,tmp,rh,ws,wd
date,,,,,,,
2023-01-01,0.017917,26.115000,18.194167,17.595652,56.725000,1.988750,198.013158
2023-01-02,0.013500,26.115000,18.194167,16.381250,56.725000,2.904583,235.478057
2023-01-03,0.012375,37.117917,24.248333,16.889474,52.979167,2.114167,225.967935


## Normalize data

In [4]:
def normalize_data(df:pd.DataFrame, pollutant:str):
    scaler_pollutant = None
    for feature in df.columns.values:
        values = df[feature].values
        values = values.reshape((-1, 1))
        scaler = StandardScaler()
        scaler = scaler.fit(values)
        normalized = scaler.transform(values)
        df[feature] = normalized.flatten()
        
        if feature == pollutant:
            scaler_pollutant = scaler    

    return df, scaler

In [23]:
data_scaler = MinMaxScaler(feature_range=(0,1))
data_norm = data_scaler.fit_transform(df_select.values)

In [26]:
data_norm[:5]

array([[0.13167801, 0.27024266, 0.98321007, 0.25369939, 0.56250986],
       [0.13167801, 0.16490893, 0.98321007, 0.43416258, 0.66898425],
       [0.20868475, 0.20899078, 0.89336398, 0.2784125 , 0.6419567 ],
       [0.44371696, 0.3435578 , 0.8205077 , 0.04655261, 0.48990875],
       [0.74056655, 0.4209641 , 0.56386168, 0.11338474, 0.41487319]])

## Timeseries to supervised learning

In [29]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg

In [30]:
data_reframed = series_to_supervised(data_norm, 1, 1)

In [ ]:
# drop columns we don't want to predict (i.e, drop all features that aren't pm25 for time t)
data_reframed.drop(data_reframed.columns[[6,7,8,9]], axis=1, inplace=True)

In [34]:
data_reframed.head(3)

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var1(t)
1,0.131678,0.270243,0.983210,0.253699,0.562510,0.131678
2,0.131678,0.164909,0.983210,0.434163,0.668984,0.208685
3,0.208685,0.208991,0.893364,0.278412,0.641957,0.443717


## Divide dataset

In [ ]:
def divide_series_by_months(df: pd.DataFrame, train=11, val:float=1):
    data_len = len(df)
    train_size = int(data_len * train)
    val_size = int(data_len * val)
    
    train_df = pd.DataFrame()
    val_df = pd.DataFrame()
    test_df = pd.DataFrame()
    
    for feature in df.columns:
        train_df[feature] = df[feature][:train_size]
        val_df[feature] = df[feature][train_size:train_size + val_size]
        test_df[feature] = df[feature][train_size + val_size:]
    

    return train_df, val_df, test_df

In [ ]:
train_df, val_df, test_df = divide_series(df, features)

In [41]:
train_df.head()

,pm25,tmp,rh,ws,wd
date,,,,,
2023-01-01,18.194167,17.595652,56.725000,1.988750,198.013158
2023-01-02,18.194167,16.381250,56.725000,2.904583,235.478057
2023-01-03,24.248333,16.889474,52.979167,2.114167,225.967935
2023-01-04,42.726250,18.440909,49.941667,0.937500,172.467173
2023-01-05,66.064167,19.333333,39.241667,1.276667,146.064584


## Search hyperparameters

## Create model

In [ ]:
class MLPOptimizer:
  """
  Class for hyperparameter optimizing for PM2.5 Forecaster.
  """
  def __init__(self, input_dim:int, n_trials:int):
    self.input_dim = input_dim
    self.n_trials = n_trials
    self.study = None
    self.X_train = None
    self.y_train = None
    self.X_val = None
    self.y_val = None

  def create_model(self, trial):
    model = Sequential()

    # Hyperparameter selection
    neurons = trial.suggest_categorical("neurons", [8, 12, 16, 24, 32, 40])
    neurons2 = trial.suggest_categorical("neurons2", [8, 12, 16, 24, 32, 40])
    dropout_rate = trial.suggest_float("dropout", 0.1, 0.2, log=True)
    # reg_type = trial.suggest_categorical("reg_type", [None])
    # reg_strength = trial.suggest_float("reg_strength", 1e-3, 1e-1, log=True)

    # Choose regularization
    #if reg_type == "l1":
     # reg = l1(reg_strength)
    #elif reg_type == "l2":
     # reg = l2(reg_strength)
    #else:
     # reg = None

    # Hidden layer
    model.add(Dense(
        neurons,
        activation="relu",
        input_dim=self.input_dim
        ))

    # Dropout layer
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1))

    # Compile model
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
    model.compile(optimizer=Adam(learning_rate), loss="mse")

    return model

  def objective(self, trial):
    model = self.create_model(trial)
    epochs = trial.suggest_categorical("epochs", [50, 100])

    # Train the model
    model.fit(
        self.X_train,
        self.y_train,
        epochs=epochs,
        validation_data=(self.X_val, self.y_val),
        verbose=0
        )

    # Evaluate on validation data
    loss = model.evaluate(self.X_val, self.y_val, verbose=0)

    return loss

  def optimize(self, X_train:np.array, y_train:np.array, X_val, y_val:np.array):
    self.X_train = X_train
    self.y_train = y_train
    self.X_val = X_val
    self.y_val = y_val

    self.study = optuna.create_study(direction="minimize")
    self.study.optimize(self.objective, n_trials = self.n_trials)

    print("Best hyperparameters:", self.study.best_params)
    return self.study

In [ ]:
mlp_optimizer = MLPOptimizer(n_steps, 32)
study = mlp_optimizer.optimize(X_train, y_train, X_val, y_val)